In [1]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:4096

In [1]:
import numpy as np
import os

from captum.attr import IntegratedGradients

import torch
from torch import nn 
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

from custom_dataset import CustomDataset
from network import Network
from utils import *

In [2]:
torch.cuda.device_count()

1

In [4]:

parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,128),nn.ReLU(), nn.Dropout(0.1),
                nn.Linear(128,2),nn.ReLU())
model = nn.DataParallel(model)
model.to(device)

# Loading the model from Job 5436878
#loading model from 6066159
load_path = os.path.join(parent_directory,'6066159','models','epoch_170')

model.load_state_dict(torch.load(load_path))

Using cuda device


<All keys matched successfully>

In [ ]:
#%%
########################
# Loading the Data #####
########################

torch.manual_seed(52)
np.random.seed(52)
# number of subprocesses to use for data loading
num_workers = 1
# how many samples per batch to load
batch_size = 5

In [ ]:

valid_data = CustomDataset(train= False,test=False)

# get filtered variables
vars = valid_data.vars

valid_sampler = SubsetRandomSampler(list(range(len(vars))))

valid_loader = DataLoader(valid_data,batch_size=batch_size, 
                            sampler= valid_sampler, num_workers=num_workers)

In [ ]:
len(vars)

In [ ]:
X,y = next(iter(valid_loader))
#X,y = X.to(device),y.to(device)

In [ ]:
y

In [ ]:
ig = IntegratedGradients(model)

In [ ]:
for X,y in valid_loader:
    X,y = X.to(device),y.to(device)
    X.requires_grad_()
    attr, delta = ig.attribute(torch.unsqueeze(X,1).float(),target=y, return_convergence_delta=True, 
                    internal_batch_size=4)
    attr = attr.detach().cpu().numpy()
    if y == 0:
        attr_0 = (attr_0 + attr)
    else:
        attr_1 = (attr_1 + attr)
    with open('attr_0.npy', 'wb') as f:
        np.save(f, attr_0)
    with open('attr_1.npy', 'wb') as f:
        np.save(f, attr_1)

# Testing the model Performance on the whole dataset

In [ ]:
actual_valid = torch.tensor([]).to(device)
pred_valid = torch.tensor([]).to(device)
for X,y in valid_loader:
    X, y = X.to(device), y.to(device)
    actual_valid = torch.cat((actual_valid,y),0)
    pred = model(torch.unsqueeze(X,1).float())
    pred_valid = torch.cat((pred_valid,torch.argmax(pred,1)),0)

In [ ]:
ConfusionMatrixDisplay.from_predictions(actual_valid.detach().cpu().numpy(),pred_valid.detach().cpu().numpy())

In [ ]:
ConfusionMatrixDisplay.from_predictions(actual_valid.detach().cpu().numpy(),pred_valid.detach().cpu().numpy())

In [ ]:
(2408 + 1356)/5469

In [ ]:
vars.new_score.value_counts()

In [ ]:
from captum.attr import visualization as viz

In [ ]:
from torch.utils.tensorboard import SummaryWriter
parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'
path = os.path.join(parent_directory,'Activations_1')
writer = SummaryWriter(log_dir=path)

In [ ]:
attr_1 = np.load('attr_1.npy')
attr_0 = np.load('attr_0.npy')

In [ ]:
attr_1.shape

In [ ]:
attr_1 = attr_1/1968
attr_0 = attr_0/3501 

In [ ]:
a1 = np.squeeze(attr_1)
a0 = np.squeeze(attr_0)

In [ ]:
fig,axes = viz.visualize_image_attr(np.expand_dims(X[65],axis=2),np.expand_dims(a1[65],axis=2),method='blended_heat_map',sign='all')

In [ ]:
for i in range(121):
    try:
        fig,axes = viz.visualize_image_attr(np.expand_dims(X[i],axis=2),np.expand_dims(attr_1[i],axis=2),method='blended_heat_map',sign = 'all')
        writer.add_figure('Slice',fig,i+1)
    except:
        print(i)

In [ ]:
fig, axes = plt.subplots(1,10)
for i,j in enumerate(range(60,70)):
    axes[i].imshow(X[:,j,:].T,cmap="gray", origin="lower")

In [ ]:
mask = np.abs(a1) > 1e-2

In [ ]:
mask.shape

In [ ]:
import json

In [ ]:
with open('region_labels.json','r') as f:
    l = json.load(f)

In [ ]:
labels = {v:{'attr_sum':0,'attrs':[]} for k,v in l.items()}

In [ ]:
from nilearn.image import load_img 
imf = load_img('/trdapps/linux-x86_64/matlab/toolboxes/spm12/tpm/labels_Neuromorphometrics.nii').get_fdata()

In [ ]:
imf = load_img('/trdapps/linux-x86_64/matlab/toolboxes/spm12/tpm/labels_Neuromorphometrics.nii').get_fdata()

In [ ]:
str(int(imf[27,102,68]))

In [ ]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        for k in range(X.shape[2]):
            if mask[i,j,k]:
                #print(i,j,k)
                r = imf[i,j,k]
                try:
                    labels[l[str(int(r))]]['attr_sum'] += np.abs(a1[i,j,k])
                    labels[l[str(int(r))]]['attrs'].append(np.abs(a1[i,j,k]))
                except KeyError:
                    print(r,(i,j,k))

In [ ]:
min(labels['Right Cerebellum Exterior']['attrs']),max(labels['Right Cerebellum Exterior']['attrs'])

In [ ]:
[(v[0],v[1]['attr_sum']) for v in sorted(labels.items(),key=lambda x: x[1]['attr_sum'],reverse=True)]

In [ ]:
labels_0 = {v:{'attr_sum':0,'attrs':[]} for k,v in l.items()}

In [ ]:
mask_0 = np.abs(a0) > 1e-2

In [ ]:
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        for k in range(X.shape[2]):
            if mask_0[i,j,k]:
                #print(i,j,k)
                r = imf[i,j,k]
                try:
                    labels_0[l[str(int(r))]]['attr_sum'] += np.abs(a0[i,j,k])
                    labels_0[l[str(int(r))]]['attrs'].append(np.abs(a0[i,j,k]))
                except KeyError:
                    print(r,(i,j,k))

In [ ]:
[(v[0],v[1]['attr_sum']) for v in sorted(labels_0.items(),key=lambda x: x[1]['attr_sum'],reverse=True)]

In [ ]:
l_cerebellum_exterior = imf == 57

In [ ]:
l_cerebellum_exterior.astype(int)

In [ ]:
affine = np.array([[  -1.5,    0. ,    0. ,   90. ],
       [   0. ,    1.5,    0. , -126. ],
       [   0. ,    0. ,    1.5,  -72. ],
       [   0. ,    0. ,    0. ,    1. ]])

In [ ]:
import nibabel as nib

In [ ]:
array_img = nib.Nifti1Image(l_cerebellum_exterior.astype(int), affine)

In [ ]:
nib.save(array_img,'r_putamen.nii')

In [ ]:
vars.new_score.value_counts()

In [ ]:
3501 + 1968

In [ ]:
from matplotlib import pyplot as plt 

plt.bar(['0','1'],[3501,1968],color=['blue','green'])
plt.xlabel('Labels')
plt.ylabel('Number of subjects')
#plt.title('Bar plot showing the distribution of lower and higher group ends')


In [ ]:
vars.score.value_counts()

In [ ]:
['green'] * 4

In [ ]:
plt.bar(['2','3','4','5','9','10','11','12'],[78,158,800,2465,1582,339,39,8],color = ['blue', 'blue', 'blue', 'blue','green', 'green', 'green', 'green' ])
plt.xlabel('Maximumm Digits Remembered')
plt.ylabel('Number of subjects')

In [ ]:
3501 + 1968